In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
import numpy as np

In [ ]:
!pip install transformers
!pip install datasets

In [ ]:
import transformers
import datasets

In [ ]:
from datasets import load_dataset

Importing data

In [ ]:
job_dataset = load_dataset('jacob-hugging-face/job-descriptions')

In [ ]:
job_dataset

DatasetDict({
    train: Dataset({
        features: ['company_name', 'job_description', 'position_title', 'description_length', 'model_response'],
        num_rows: 853
    })
})

In [ ]:
length=853

In [ ]:
set(job_dataset['train']['position_title'])

taking four job description from software related

In [ ]:
job_roles=['Software Developer','Software Engineer']

taking only
1.job_description 2.position_title

In [ ]:
job_des=[]
pos_title=[]
indices=[]

In [ ]:
#getting 2 job description of any position_tile

def get_job_des_pos(cat,job_des,pos_title):
    count=0

    for i in range(length):
        if job_dataset['train']['position_title'][i]==cat:
            job_des.append(job_dataset['train']['job_description'][i])
            pos_title.append(job_dataset['train']['position_title'][i])
            count+=1
            indices.append(i)
            if count==2:
                break



In [ ]:
get_job_des_pos('Software Developer',job_des,pos_title)

In [ ]:
len(job_des)

2

In [ ]:
pos_title

['Software Developer', 'Software Developer']

In [ ]:
get_job_des_pos('Software Engineer',job_des,pos_title)


In [ ]:
pos_title

['Software Developer',
 'Software Developer',
 'Software Engineer',
 'Software Engineer']

In [ ]:
indices

[94, 95, 30, 33]

selecting 15 job descriptions index numbers

In [ ]:
l=list(np.random.randint(0,length, size=(15, )))

In [ ]:
l

[137, 655, 311, 695, 159, 670, 836, 152, 257, 105, 85, 351, 797, 322, 426]

In [ ]:
count=len(job_des)
for i in l:
    #to avoid already taken job descriptions
    if i not in indices:
            job_des.append(job_dataset['train']['job_description'][i])
            pos_title.append(job_dataset['train']['position_title'][i])

            count+=1
            if count==15:
                break


In [ ]:
len(job_des),len(pos_title)

(15, 15)

In [ ]:
job_df=pd.DataFrame({'job_description':job_des,'position_title':pos_title})

In [ ]:
job_df.head()

,job_description,position_title
0,were not looking for rockstars zombies or magi...,Software Developer
1,at remoteteamcom we are building an hr tool fo...,Software Developer
2,tldr\n\nhighimpact software development jobs f...,Software Engineer
3,description hughes private capital in business...,Software Engineer
4,about bethany\n\nfor years bethany has been i...,Registered Nurse (RN-CA02-007)


preprocess the job description and generate tokens

In [ ]:
import re
import spacy
nlp=spacy.load('en_core_web_sm')

In [ ]:
def preprocess(text):
    # convert all characters in the string to lower case
    text = re.sub(r'\n', ' ', text)

    #remove digits
    text = re.sub(r'\d', '', text)

    # Remove punctuation (you can customize this pattern for your needs)
    text = re.sub(r'[^\w\s]', '', text)

    # Remove extra spaces and replace with a single space
    text = re.sub(r'\s+', ' ', text).strip()

    # tokenize word
    doc=nlp(text)
    toks=[i.text.lower() for i in doc if not i.is_stop]


    return "," .join(toks)

In [ ]:
job_df['tokens']=job_df['job_description'].apply(preprocess)

In [ ]:
job_df.head()

,job_description,position_title,tokens
0,were not looking for rockstars zombies or magi...,Software Developer,"looking,rockstars,zombies,magicians,looking,mo..."
1,at remoteteamcom we are building an hr tool fo...,Software Developer,"remoteteamcom,building,hr,tool,companies,remot..."
2,tldr\n\nhighimpact software development jobs f...,Software Engineer,"tldr,highimpact,software,development,jobs,smar..."
3,description hughes private capital in business...,Software Engineer,"description,hughes,private,capital,business,fa..."
4,about bethany\n\nfor years bethany has been i...,Registered Nurse (RN-CA02-007),"bethany,years,bethany,improving,lives,alberta,..."


getting skills from the tokens for each row

In [ ]:
!python -m spacy download en_core_web_lg


2023-09-18 11:56:01.958180: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 587.7/587.7 MB 927.2 kB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_lg')


In [ ]:
import spacy
nlp = spacy.load("en_core_web_lg")
skill_pattern_path = "/content/drive/MyDrive/Colab Notebooks/resume skills - assessment/JsonFiles/total_skills2.jsonl"

adding new entity(to spacy pipeline)

In [ ]:
ruler = nlp.add_pipe("entity_ruler")
ruler.from_disk(skill_pattern_path)
nlp.pipe_names

['tok2vec',
 'tagger',
 'parser',
 'attribute_ruler',
 'lemmatizer',
 'ner',
 'entity_ruler']

In [ ]:
def get_skills_tokens(tokens):
    tokens=tokens.split(',')
    text=' '.join(tokens)
    doc = nlp(text)
    skill_set=set()
    for ent in doc.ents:
        if ent.label_ == "SKILL":
            skill_set.add(ent.text)
    return ','.join(list(skill_set))

In [ ]:
job_df['skills']=job_df['tokens'].apply(get_skills_tokens)

In [ ]:
job_df['skills_count']=job_df['tokens'].apply(lambda x:len(x.split(",")))

In [ ]:
job_df.head()

,job_description,position_title,tokens,skills,skills_count
0,were not looking for rockstars zombies or magi...,Software Developer,"looking,rockstars,zombies,magicians,looking,mo...","schedules,support,javascript,business,vision,r...",373
1,at remoteteamcom we are building an hr tool fo...,Software Developer,"remoteteamcom,building,hr,tool,companies,remot...","business,ability,think,engineering,ruby,react,...",127
2,tldr\n\nhighimpact software development jobs f...,Software Engineer,"tldr,highimpact,software,development,jobs,smar...","ability,software engineering,marathon,library,...",316
3,description hughes private capital in business...,Software Engineer,"description,hughes,private,capital,business,fa...","release,engineering,throughput,bash,support,ja...",696
4,about bethany\n\nfor years bethany has been i...,Registered Nurse (RN-CA02-007),"bethany,years,bethany,improving,lives,alberta,...","learner,ability,certificate,leadership,deliver...",444


In [ ]:
job_df['skills_count']

0     373
1     127
2     316
3     696
4     444
5     236
6     452
7     346
8     757
9     181
10    321
11    371
12    298
13    538
14    163
Name: skills_count, dtype: int64

Save the csv file

In [ ]:
job_df.to_csv('/content/drive/MyDrive/Colab Notebooks/resume skills - assessment/CsvFiles/job_df.csv',index=False)